<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Mohammad Abolnejadian
### Student Number : 98103867
### Colab Link: https://colab.research.google.com/drive/1iG9PjhNA7T1N5DOV0z-4ng6hhEYyWSaJ?usp=sharing
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [35]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-07 19:42:11--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-07 19:42:11--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0b55ae49bd29bdc8bc54c7ff58.dl.dropboxusercontent.com/cd/0/get/B7n1_t9ZzCuNm8TE3BujWpeMIUS5tBQzuawLw4-DTPmotLdsuq7Gp53i5C-qbGZHGirK5Zob3gQLIw7hZFdUDmnGldMWN1uwwdMNEIWlj_S5K55XHHPAxxae16V1HXle2foQV-2dZIRd7wM-fZ1G0JTzLyI9ssziBlaMYZhZ1qqAVuoOZQmSAK9heKEXo3DLGb4/file?dl=1# [following]
--2023-05-07 19:42:12--  https://uc0b55ae49bd29bdc8bc54c7ff58.dl.dropboxusercontent.com/cd/0/get/B7n1_t9ZzCuNm8TE3BujWpeMIUS5tBQzuawLw4-DTP

In [36]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

mkdir: cannot create directory ‘/content/dataset/’: File exists
Archive:  /content/dataset.zip
replace /content/dataset/content/news-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/dataset/content/news-train.csv  


# Preprocessing

In [37]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [38]:
df = pd.read_csv('/content/dataset/content/news-train.csv')
df

,topic,text
0,sports,به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1,politics,پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2,politics,# دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3,economics,کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4,economics,به گزارش ایسنا صحبت با برخی # # آن است که امرو...
...,...,...
197356,sports,به گزارش ورزش سه و به نقل از مارکا رقابت‌های ب...
197357,sports,به گزارش ایسنا و به نقل از فوتبال ایتالیا یوون...
197358,sports,سرمربی تیم والیبال دانشجویان گفت ما اصلا فرصت ...
197359,sports,به گزارش ورزش سه مرتضی # باشگاه # مصر # را نسب...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [39]:
topic_count = df.groupby(['topic']).count()
total_number_of_news = topic_count['text'].sum()
print("Total number of news: " + str(total_number_of_news))
topic_count

Total number of news: 197361


,text
topic,
cultural,13359
economics,49600
politics,11297
sports,123105


Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [40]:
downsampled_df = pd.DataFrame()
min_number_of_news_for_topic = topic_count['text'].min()

for topic in topic_count.index:
  data = df[df['topic'] == topic]
  downsampled_data = data.sample(min_number_of_news_for_topic)
  downsampled_df = pd.concat([downsampled_df, downsampled_data])

print(downsampled_df)

print("########")

downsampled_topic_count = downsampled_df.groupby(['topic']).count()
print(downsampled_topic_count)

           topic                                               text
46562   cultural  نشست روسای دانشگاه‌های برتر و بزرگ کشور با نما...
153376  cultural  # همکاری بین موسسه فرهنگی # اداره کل فرهنگ و ا...
125601  cultural  دکتر # صالحی وزیر فرهنگ و ارشاد اسلامی در پاسخ...
76182   cultural  این تهیه‌کننده و # که با ایسنا گفت‌وگو می‌کرد ...
190655  cultural  سومین دوره از هفته سینمایی خانه آسیا با شرکت آ...
...          ...                                                ...
76766     sports  به گزارش ورزش سه در شرایطی که طبق برنامه قبلی ...
123933    sports  به گزارش ایسنا در ادامه هفته سی و چهارم رقابت‌...
159084    sports  مسئولین فدراسیون والیبال با حضور در کمیته ملی ...
7209      sports  به گزارش ورزش سه عیسی آل # درباره دیدار برابر ...
3132      sports  به گزارش ورزش سه قهرمانی فرانسه در جام جهانی ر...

[45188 rows x 2 columns]
########
            text
topic           
cultural   11297
economics  11297
politics   11297
sports     11297


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [41]:
train_df = downsampled_df.sample(frac=0.8, random_state=1)
test_df = downsampled_df.drop(train_df.index)
print("train df: ")
print(train_df)
print("test_df: ")
print(test_df)

train df: 
            topic                                               text
135686     sports  به گزارش ایسنا رییس فدراسیون وزنه‌برداری آمریک...
72770      sports  به گزارش ورزش سه جلسه هیات مدیره باشگاه استقلا...
112894   politics  حسین # نایب رئیس اول کمیسیون امنیت ملی دولت پا...
140642  economics  قیمت طلا در بازارهای بین‌المللی در آغاز مبادلا...
103680     sports  به گزارش ورزش سه محمد # که همواره با صراحت # ش...
...           ...                                                ...
54915      sports  # مهرماه # اولین تجربه مربیگری علی دایی بود \n...
43916   economics  رئیس بانک مرکزی گفت تصمیم اخیر دولت در تعیین ن...
130847     sports  به‌ گزارش ‌ایسنا ‌و ‌به نقل از # منچسترسیتی در...
187117   politics  اهتمام فراوان روحانی به پروتکل‌های بهداشتی به ...
93804   economics  طبق بررسی‌های انجام شده و # جهانی وجود بازار پ...

[36150 rows x 2 columns]
test_df: 
           topic                                               text
78940   cultural  به گزارش ایسنا بر اساس خبر رسیده همایون

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [42]:
train_df['tokens'] = train_df['text'].str.strip().str.split(' ')
train_df

,topic,text,tokens
135686,sports,به گزارش ایسنا رییس فدراسیون وزنه‌برداری آمریک...,"[به, گزارش, ایسنا, رییس, فدراسیون, وزنه‌برداری..."
72770,sports,به گزارش ورزش سه جلسه هیات مدیره باشگاه استقلا...,"[به, گزارش, ورزش, سه, جلسه, هیات, مدیره, باشگا..."
112894,politics,حسین # نایب رئیس اول کمیسیون امنیت ملی دولت پا...,"[حسین, #, نایب, رئیس, اول, کمیسیون, امنیت, ملی..."
140642,economics,قیمت طلا در بازارهای بین‌المللی در آغاز مبادلا...,"[قیمت, طلا, در, بازارهای, بین‌المللی, در, آغاز..."
103680,sports,به گزارش ورزش سه محمد # که همواره با صراحت # ش...,"[به, گزارش, ورزش, سه, محمد, #, که, همواره, با,..."
...,...,...,...
54915,sports,# مهرماه # اولین تجربه مربیگری علی دایی بود \n...,"[#, مهرماه, #, اولین, تجربه, مربیگری, علی, دای..."
43916,economics,رئیس بانک مرکزی گفت تصمیم اخیر دولت در تعیین ن...,"[رئیس, بانک, مرکزی, گفت, تصمیم, اخیر, دولت, در..."
130847,sports,به‌ گزارش ‌ایسنا ‌و ‌به نقل از # منچسترسیتی در...,"[به‌, گزارش, ‌ایسنا, ‌و, ‌به, نقل, از, #, منچس..."
187117,politics,اهتمام فراوان روحانی به پروتکل‌های بهداشتی به ...,"[اهتمام, فراوان, روحانی, به, پروتکل‌های, بهداش..."


Count the tokens and list the top 5 frequent ones (5 points)

In [43]:
train_df_exploded = train_df.explode('tokens')
top_5 = train_df_exploded['tokens'].value_counts().nlargest(5)
top_5_df = pd.DataFrame({'token': top_5.index, 'frequency': top_5.values})
top_5_df

,token,frequency
0,#,1632298
1,و,879644
2,در,744791
3,\n,596108
4,به,584292


Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [44]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-07 19:43:06--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-07 19:43:06--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-07 19:43:06 (36.4 MB/s)

In [45]:
#read stopwords
stopwords_file = 'stopwords.txt'
with open(stopwords_file) as f:
    stopwords = set(f.read().splitlines())
stopwords.update(['#', '\n'])
#remove from text
train_df['text'] = train_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
#remove from tokens
train_df['tokens'] = train_df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])
train_df

,topic,text,tokens
135686,sports,گزارش ایسنا رییس فدراسیون وزنه‌برداری آمریکا ا...,"[گزارش, ایسنا, رییس, فدراسیون, وزنه‌برداری, آم..."
72770,sports,گزارش ورزش سه جلسه هیات مدیره باشگاه استقلال ش...,"[گزارش, ورزش, سه, جلسه, هیات, مدیره, باشگاه, ا..."
112894,politics,حسین نایب رئیس کمیسیون امنیت ملی دولت پاکستان ...,"[حسین, نایب, رئیس, کمیسیون, امنیت, ملی, دولت, ..."
140642,economics,قیمت طلا بازارهای بین‌المللی آغاز مبادلات دیرو...,"[قیمت, طلا, بازارهای, بین‌المللی, آغاز, مبادلا..."
103680,sports,گزارش ورزش سه محمد صراحت شناخته می‌شود تصمیمات...,"[گزارش, ورزش, سه, محمد, صراحت, شناخته, می‌شود,..."
...,...,...,...
54915,sports,مهرماه اولین تجربه مربیگری علی دایی جدایی آلما...,"[مهرماه, اولین, تجربه, مربیگری, علی, دایی, جدا..."
43916,economics,رئیس بانک مرکزی تصمیم اخیر دولت تعیین نرخ ارز ...,"[رئیس, بانک, مرکزی, تصمیم, اخیر, دولت, تعیین, ..."
130847,sports,به‌ گزارش ‌ایسنا ‌و ‌به نقل منچسترسیتی این فصل...,"[به‌, گزارش, ‌ایسنا, ‌و, ‌به, نقل, منچسترسیتی,..."
187117,politics,اهتمام فراوان روحانی پروتکل‌های بهداشتی نظر می...,"[اهتمام, فراوان, روحانی, پروتکل‌های, بهداشتی, ..."


Make a list of the remaining unique tokens (5 points)

In [46]:
train_df_exploded = train_df.explode('tokens')
unique_tokens = train_df_exploded['tokens'].unique()
number_of_unique_tokens = unique_tokens.size
print(unique_tokens.tolist())
print("number of unique tokens: " + str(number_of_unique_tokens))

['گزارش', 'ایسنا', 'رییس', 'فدراسیون', 'وزنه\u200cبرداری', 'آمریکا', 'اقدامات', 'جهانی', 'برای', 'مقابله', 'دوپینگ', 'تمجید', 'کرد', 'گفت\u200c', 'تیم\u200c', 'اعتماد', 'داریم', 'سریع', 'شفاف', 'چندی', 'جلسه', 'هیات', 'اجرایی', 'آخر', 'ماه', 'دسامبر', 'مهلت', 'مشکلات', 'مربوط', 'حل', 'کند', 'این\u200c', 'راستا', 'نیز', 'تشکیل', 'آ', 'حمایت', 'مطمئن', 'هستم', 'فردی', 'دوباره', 'پنجمین', 'بهترین', 'فرد', 'مبارزه', 'تیمی', 'سوی', 'شده\u200cاند', 'می\u200cکنیم', 'حاضر', 'کار', 'زیادی', 'حوزه', 'آموزش', 'تست', 'گرفتن', 'باید', 'این', 'گروه', 'کنیم', 'فرهنگ', 'ورزش', 'تغییر', 'سه', 'مدیره', 'باشگاه', 'استقلال', 'شب', 'گذشته', 'حضور', 'اعضا', 'برگزار', 'مدیران', 'مسائل', 'بحث', 'تبادل', 'نظر', 'پرداختند', 'اگرچه', 'می\u200cشد', 'شاید', 'دیشب', 'انتخاب', 'مدیرعامل', 'جدید', 'تصمیم', 'مساله', 'اصلی', 'شکایت', 'آندره', 'استراماچونی', 'مربی', 'ایتالیایی', 'توانسته', 'حکم', 'فیفا', 'دریافت', 'مبلغی', 'حدود', 'میلیون', 'یورو', 'محکوم', 'کرده', 'حالا', 'فکر', 'مشورت', 'بخش', 'بین\u200cالملل', 'قصد',

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [47]:
vectorizer = CountVectorizer(vocabulary=unique_tokens)
term_doc_matrix = vectorizer.fit_transform(train_df['text'])
x_train = term_doc_matrix.toarray()
x_train

array([[1, 1, 2, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [48]:
###Custom solution to this problem adding a new column to the datafram

# from collections import Counter

# # def count_occurrences(lst):
# #     return {i: lst.count(i) for i in lst}

# train_df['occ'] = train_df['tokens'].apply(lambda x: dict(Counter(x)))


# train_df['x_train'] = train_df['occ'].apply(lambda x: [x.get(token, 0) for token in unique_tokens])
# # df = df.drop(['new_col_1', ], axis=1, errors='ignore')
# train_df = train_df.drop('occ', axis=1)
# train_df

In [49]:
#Check the sanity of our solution
# for token in unique_tokens[:10]:
#   print(token)
# Counter(train_df.at[165543, 'tokens'])

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [50]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_df['topic'])
y_train

array([3, 3, 2, ..., 3, 2, 1])

In [51]:
#Another way with pandas factorize
# train_df['y_train'] = pd.factorize(train_df['topic'])[0]
# train_df
# x_train = train_df['x_train'].values
# y_train = train_df['y_train'].values
# x_train.size

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [52]:
###preprocess test data
#tokenize
test_df['tokens'] = test_df['text'].str.strip().str.split(' ')
#remove stopwords from text
test_df['text'] = test_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
#remove stopwords from tokens
test_df['tokens'] = test_df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])

In [53]:
vectorizer = CountVectorizer(vocabulary=unique_tokens)
term_doc_matrix = vectorizer.fit_transform(test_df['text'])
x_test = term_doc_matrix.toarray()
x_test

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [54]:
###generate x_test(custom)
# test_df['occ'] = test_df['tokens'].apply(lambda x: dict(Counter(x)))
# test_df['x_test'] = test_df['occ'].apply(lambda x: [x.get(token, 0) for token in unique_tokens])
# test_df = test_df.drop('occ', axis=1)
# test_df

#iterate through rows and add x_train
# default_list = [0] * number_of_unique_tokens
# test_df['x_test'] = pd.Series([list(default_list) for x in range(len(df.index))])
# for index, row in test_df.iterrows():
#     # row['x_train'] = [0] * number_of_unique_tokens
#     token_occurences = dict(Counter(row['tokens']))
#     for token, occ in token_occurences.items():
#       tokens_idx = mapping_token_to_index[token]
#       row['x_test'][tokens_idx] = occ
# test_df

In [55]:
encoder = LabelEncoder()
y_test = encoder.fit_transform(test_df['topic'])
y_test

array([0, 0, 0, ..., 3, 3, 3])

In [56]:
###Turning columns into numpy arrays (in the custom solution)
# x_test = test_df['x_test'].values
# y_test = test_df['y_test'].values
# x_test.size

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [57]:
clf = MultinomialNB(force_alpha=True)
clf.fit(x_train, y_train)

MultinomialNB(force_alpha=True)

Get the predictions of the model for the testing news (`x_test`).

In [58]:
sklearn_predictions = clf.predict(x_test)
sklearn_predictions

array([0, 0, 0, ..., 3, 3, 3])

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [59]:
print(classification_report(y_test, sklearn_predictions))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      2246
           1       0.91      0.93      0.92      2198
           2       0.95      0.90      0.92      2280
           3       0.99      0.98      0.99      2314

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [60]:
#We use gaussian naive bayes
import math
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        """
        YOUR CODE (IF NECESSARY)
        """
        self.classes = None
        self.number_of_classes = None
        self.priors = dict()
        self.mean = dict()
        self.var = dict()
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        #calculate initial values
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        self.number_of_classes = len(self.classes)
        
        for c in self.classes:
            x_c = X[y == c]
            self.priors[c] = x_c.shape[0] / float(n_samples)
            self.mean[c] = x_c.mean(axis=0)
            self.var[c] = x_c.var(axis=0)    
        return self
    
    def normal_pdf(self, c, x):
        eps = 1e-3
        mean = self.mean[c]
        var = self.var[c] + eps
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


    def new_normale_pdf(self, mean, var, x):
        eps = 1e-2 # Added in denominator to prevent division by zero
        coeff = 1.0 / math.sqrt(2.0 * math.pi * var + eps)
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * var + eps)))
        return coeff * exponent

    def classify(self, x):
      posteriors = []
      for c in self.classes:
        prior = np.log(self.priors[c])
        class_pdfs = np.sum(np.log(self.normal_pdf(c, x)))
        posterior = prior + class_pdfs
        posteriors.append(posterior)

      return self.classes[np.argmax(posteriors)]


    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        y_pred = [self.classify(x) for x in X]
        """
        YOUR CODE
        """
        return y_pred

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [61]:
custom_model = Naive_Bayes()
custom_model.fit(x_train, y_train)

Get the predictions of your model for the testing news (`x_test`).

In [62]:
custom_model_predictions = custom_model.predict(x_test)
custom_model_predictions

<ipython-input-60-7a914d68d15e>:62: RuntimeWarning: divide by zero encountered in log
  class_pdfs = np.sum(np.log(self.normal_pdf(c, x)))


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,


Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [63]:
print(classification_report(y_test, custom_model_predictions))

              precision    recall  f1-score   support

           0       0.69      0.93      0.79      2246
           1       0.83      0.80      0.81      2198
           2       0.90      0.61      0.73      2280
           3       0.88      0.89      0.88      2314

    accuracy                           0.81      9038
   macro avg       0.82      0.81      0.80      9038
weighted avg       0.82      0.81      0.80      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef